In [7]:
# load libraries
import pandas as pd
import os

# read files
excel_file = "../data/Murine_Protein_Solubility_Data/Demontis example_Up & down proteins for proteome enrichment.xlsx"
sheets = ['up', 'down']
output_folder = '../data/Murine_Protein_Solubility_Data/processed/' 
os.makedirs(output_folder, exist_ok=True)

dfs = []
for sheet in sheets:
    # read sheet
    df = pd.read_excel(excel_file, sheet_name=sheet)
    df.columns = df.columns.str.strip()
    # split GN_ID into parts
    split_cols = df['GN_ID'].str.split('|', expand=True)
    df['Prefix'] = split_cols[0]
    df['UniProt_ID'] = split_cols[1]
    df['UniProt_Name'] = split_cols[2]
    # add direction col
    df['Direction'] = sheet
    # save
    dfs.append(df)

# combine up and down sheets
all_proteins = pd.concat(dfs, ignore_index=True)
print(all_proteins.head(3))
all_proteins.to_csv(os.path.join(output_folder, 'all_enriched_proteins.csv'), index=False)

# create a demo subset for test (N=20 for each direction)
topN = 20
log2fc_threshold = 1
log10pval_threshold = 1.5

up_proteins = (
    all_proteins[
        (all_proteins['Direction'] == 'up') &
        (all_proteins['log2FC(insoluble/soluble protein ratio)'] > log2fc_threshold) &
        (all_proteins['-LOG10(P-VALUE)'] > log10pval_threshold)
    ]
    .nlargest(topN, '-LOG10(P-VALUE)')
)

down_proteins = (
    all_proteins[
        (all_proteins['Direction'] == 'down') &
        (all_proteins['log2FC(insoluble/soluble protein ratio)'] < -log2fc_threshold) &
        (all_proteins['-LOG10(P-VALUE)'] > log10pval_threshold)
    ]
    .nlargest(topN, '-LOG10(P-VALUE)')
)

# concat list and save 
demo_proteins = pd.concat([up_proteins, down_proteins])
print(demo_proteins.shape) 
demo_proteins.to_csv(os.path.join(output_folder, 'demo_proteins_for_download.csv'), index=False)
demo_proteins[['UniProt_ID']].to_csv(os.path.join(output_folder, 'uniprot_ids_for_batch.txt'), index=False, header=False)

                          GN_ID Gene_Name  \
0   Apoa2_sp|P09813|APOA2_MOUSE     Apoa2   
1     Hrnr_sp|Q8VHD8|HORN_MOUSE      Hrnr   
2  Mup10_tr|A2BIN1|A2BIN1_MOUSE     Mup10   

   log2FC(insoluble/soluble protein ratio)  -LOG10(P-VALUE)    Prefix  \
0                                 3.319557         1.927557  Apoa2_sp   
1                                 2.918873         1.789432   Hrnr_sp   
2                                 2.596492         2.066665  Mup10_tr   

  UniProt_ID  UniProt_Name Direction  
0     P09813   APOA2_MOUSE        up  
1     Q8VHD8    HORN_MOUSE        up  
2     A2BIN1  A2BIN1_MOUSE        up  
(40, 8)


## SCanchi Notes

The above script reads the original excel sheet, extracts the data for both up and down proteins, splits the column that has uniprot ids and creates a demo protein list (top 20 for either direction) to be used for initial data acquisition. 

The next step is to obtain both the sequence data and structure data for the protein subset. UniProt offers [programmatic access](https://www.uniprot.org/api-documentation/idmapping) and includes mapping to other databases. However, I found out that the mapping does not actually download the files and that is a separate step. Since we need structure information and it is unlikely for PDB to have all, as a first step, I plan to get predicted structure from AlphaFold. Cross checking PDB for protein in the list for solved structres to compare against predicted structure from Alphafold can become an optional downstream step. Since AlphaFold offers the model organism proteome, choosing to download the entire set for local query instead of API calls for individual structures.      

In [13]:
# load libraries
import os
import requests

# set paths
base_dir = '../data/Murine_Protein_Solubility_Data/processed'
id_file = os.path.join(base_dir, 'uniprot_ids_for_batch.txt')
output_fasta = os.path.join(base_dir, 'demo_protein_sequences.fasta')

# read the ids from file
with open(id_file) as f:
    ids = [line.strip().split('-')[0] for line in f if line.strip()]
ids = list(set(ids))    

# get ids as comma separated for api call
query = " OR ".join([f"accession:{x}" for x in ids])
url = f"https://rest.uniprot.org/uniprotkb/stream?format=fasta&query=({query})"
print("Querying UniProt with URL:")
print(url)
response = requests.get(url)

if response.status_code == 200:
    with open(output_fasta, 'w') as fasta_out:
        fasta_out.write(response.text)
    print(f"FASTA seqeunces saved to {output_fasta}")
else:
    print(f"Download failed with status code: {response.status_code}")

Querying UniProt with URL:
https://rest.uniprot.org/uniprotkb/stream?format=fasta&query=(accession:P14069 OR accession:Q8C3X2 OR accession:P56812 OR accession:P22599 OR accession:P49817 OR accession:O08917 OR accession:P11588 OR accession:P11798 OR accession:E9QA79 OR accession:Q9Z2L6 OR accession:Q8BFZ9 OR accession:A0A075B6A3 OR accession:P56376 OR accession:Q8VHD8 OR accession:O70475 OR accession:Q91X78 OR accession:P07758 OR accession:Q9D0M1 OR accession:P01592 OR accession:Q00896 OR accession:P07759 OR accession:P97352 OR accession:P03977 OR accession:Q91W29 OR accession:P63300 OR accession:A2AQ43 OR accession:P09813 OR accession:Q64314 OR accession:A2BIN1 OR accession:Q9Z239 OR accession:Q6PD31 OR accession:Q8R574 OR accession:P12246 OR accession:Q00898 OR accession:P50543)
FASTA seqeunces saved to ../data/Murine_Protein_Solubility_Data/processed/demo_protein_sequences.fasta


# SCanchi Notes

I downloaded the latest mouse proteome from AlphaFold: `wget https://ftp.ebi.ac.uk/pub/databases/alphafold/latest/UP000000589_10090_MOUSE_v4.tar -P af_mouse_latest/`

For the UniProt download, I initially ran in error code 400 (bad URL). Upon closer inspection, it turned out some of the UniProt IDs had isoform format (-1, -2 etc) which probably doesn't work for query URL. There were also duplicate entries between the up and down sets which most likely stems from collapsing of data across multiple experiments/groups etc. 

The next step is to run Aggrescan3D which helps calculate the aggregation propensity score given a protein structure. The reason the team decided to use the 2.0 version was since the latest 4.0 version includes pH aware calculation which might time for completion. I looked into the [documentation](https://biocomp.chem.uw.edu.pl/A3D2/) and they have both a local download and a rest API access. The local download requires python 2.7 but since their REST API is well documented, I decided to use that instead. Since the Aggrescan3D requires pdb files in uncompressed format, I decided to create a separate folder for the subset files. This also allowed to capture any missing structure files in the AF reference side. 

In [14]:
# load libraries
import os
import shutil

# set paths
base_dir = '../data/Murine_Protein_Solubility_Data/processed'
all_af = '../data/Murine_Protein_Solubility_Data/af_mouse_latest'
id_file = os.path.join(base_dir, 'uniprot_ids_for_batch.txt')
demo_dir = os.path.join(base_dir, 'af_demo')
os.makedirs(demo_dir, exist_ok=True)

# read the ids from file
with open(id_file) as f:
    ids = [line.strip().split('-')[0] for line in f if line.strip()]
ids = list(set(ids))  

# copy matching .pdb.gz files
copied = 0
for acc in ids:
    af_filename = f"AF-{acc}-F1-model_v4.pdb.gz"
    src = os.path.join(all_af, af_filename)
    dst = os.path.join(demo_dir, af_filename)
    if os.path.exists(src):
        shutil.copy(src, dst)
        copied += 1
        print(f"Copied {af_filename}")
    else:
        print(f"Not found: {af_filename}")
print(f"Copied {copied} files to {demo_dir}")

Copied AF-P14069-F1-model_v4.pdb.gz
Copied AF-Q8C3X2-F1-model_v4.pdb.gz
Copied AF-P56812-F1-model_v4.pdb.gz
Copied AF-P22599-F1-model_v4.pdb.gz
Copied AF-P49817-F1-model_v4.pdb.gz
Copied AF-O08917-F1-model_v4.pdb.gz
Copied AF-P11588-F1-model_v4.pdb.gz
Copied AF-P11798-F1-model_v4.pdb.gz
Copied AF-E9QA79-F1-model_v4.pdb.gz
Copied AF-Q9Z2L6-F1-model_v4.pdb.gz
Copied AF-Q8BFZ9-F1-model_v4.pdb.gz
Not found: AF-A0A075B6A3-F1-model_v4.pdb.gz
Copied AF-P56376-F1-model_v4.pdb.gz
Copied AF-Q8VHD8-F1-model_v4.pdb.gz
Copied AF-O70475-F1-model_v4.pdb.gz
Copied AF-Q91X78-F1-model_v4.pdb.gz
Copied AF-P07758-F1-model_v4.pdb.gz
Copied AF-Q9D0M1-F1-model_v4.pdb.gz
Copied AF-P01592-F1-model_v4.pdb.gz
Copied AF-Q00896-F1-model_v4.pdb.gz
Copied AF-P07759-F1-model_v4.pdb.gz
Copied AF-P97352-F1-model_v4.pdb.gz
Copied AF-P03977-F1-model_v4.pdb.gz
Copied AF-Q91W29-F1-model_v4.pdb.gz
Not found: AF-P63300-F1-model_v4.pdb.gz
Not found: AF-A2AQ43-F1-model_v4.pdb.gz
Copied AF-P09813-F1-model_v4.pdb.gz
Copied AF-Q6

In [21]:
# load libraries
import os
import time
import json
import glob
import requests
import numpy as np
import pandas as pd

# set paths
base_dir = '../data/Murine_Protein_Solubility_Data/processed'
pdb_dir = os.path.join(base_dir, 'af_demo')
output_summary = os.path.join(base_dir, 'demo_aggrescan_scores_batches.csv')
failure_log = os.path.join(base_dir, "demo_aggrescan_failure_log.txt")

# calculate batch size and batch pdb files
pdb_files = sorted(glob.glob(os.path.join(pdb_dir, "*.pdb")))
num_batches = 4
batches = np.array_split(pdb_files, num_batches)
print(f"Total PDB files: {len(pdb_files)}")
for i, batch in enumerate(batches):
    print(f"Batch {i+1}: {len(batch)} files")

all_results = []

# loop through each batch
for batch_idx, batch in enumerate(batches):
    print(f"\nProcessing batch {batch_idx+1} of {len(batches)}")
    for pdb_path in batch:
        pdb_file = os.path.basename(pdb_path)
        options = {
            'dynamic': False,
            'distance': 5,
            'email': None,
            'name': os.path.splitext(pdb_file)[0],
            'hide': True,
            'foldx': True,
        }
        # submit job to Aggrescan3D REST API
        with open(pdb_path, 'rb') as fh:
            files = {
                'inputfile': (pdb_file, fh),
                'json': (None, json.dumps(options), 'application/json')
            }
            print(f"Submitting {pdb_file}...", end='')
            req = requests.post(
                'https://biocomp.chem.uw.edu.pl/A3D2/RESTful/submit/userinput/',
                files=files
            )
        if req.status_code != 200:
            print(f"[Failed: {req.status_code}]")
            all_results.append({'pdb': pdb_file, 'status': 'submission_failed'})
            continue
        jobid = req.json()['jobid']
        print(f" submitted as jobid={jobid}. Waiting for completion...", end='')
        
        # poll for job completion (based on jobid status)
        while True:
            stat = requests.get(f'https://biocomp.chem.uw.edu.pl/A3D2/RESTful/job/{jobid}/status/')
            status_str = stat.text.strip()
            if 'done' in status_str:
                print(" Done.")
                break
            elif 'error' in status_str:
                print(f" Error: {status_str}")
                all_results.append({'pdb': pdb_file, 'jobid': jobid, 'status': 'error'})
                break
            else:
                print(".", end='', flush=True)
                time.sleep(15)  

        # get the aggregation scores if done
        if 'done' in status_str:
            r = requests.get(f'https://biocomp.chem.uw.edu.pl/A3D2/RESTful/job/{jobid}/')
            try:
                data = r.json()
                # added support for key "A3Dscore"
                if 'A3Dscore' in data:
                    a3d = data['A3Dscore']
                elif 'aggrescan3Dscore' in data:
                    a3d = data['aggrescan3Dscore']
                else:
                    raise KeyError("No A3Dscore or aggrescan3Dscore in data")
                all_results.append({
                    'pdb': pdb_file,
                    'jobid': jobid,
                    'status': 'done',
                    'avg': a3d.get('avg', a3d.get('average_value', 'NA')),
                    'sum': a3d.get('sum', a3d.get('summary_value', 'NA')),
                    'min': a3d.get('min', a3d.get('min_value', 'NA')),
                    'max': a3d.get('max', a3d.get('max_value', 'NA')),
                })
            except Exception as ex:
                print(f"\nError parsing result for {jobid} ({pdb_file}): {ex}")
                with open(failure_log, "a") as log:
                    log.write(f"{pdb_file}  |  jobid: {jobid}\n")
                    log.write(r.text)
                    log.write("\n\n" + "="*60 + "\n\n")
                all_results.append({'pdb': pdb_file, 'jobid': jobid, 'status': 'result_parse_failed'})

# save results
df = pd.DataFrame(all_results)
df.to_csv(output_summary, index=False)
print(f"\nSaved results to {output_summary}")


Total PDB files: 32
Batch 1: 8 files
Batch 2: 8 files
Batch 3: 8 files
Batch 4: 8 files

Processing batch 1 of 4
Submitting AF-A2BIN1-F1-model_v4.pdb... submitted as jobid=33d547779e8a66f. Waiting for completion.............. Done.
Submitting AF-E9QA79-F1-model_v4.pdb... submitted as jobid=26290e19382496d. Waiting for completion............... Done.
Submitting AF-O08917-F1-model_v4.pdb... submitted as jobid=b4adcf8900ae312. Waiting for completion................................... Done.
Submitting AF-O70475-F1-model_v4.pdb... submitted as jobid=a8b3fa6843d54a8. Waiting for completion.................................. Done.
Submitting AF-P01592-F1-model_v4.pdb... submitted as jobid=77534d5946deb9e. Waiting for completion.......... Done.
Submitting AF-P03977-F1-model_v4.pdb... submitted as jobid=c09fa2892fb5896. Waiting for completion....... Done.
Submitting AF-P07758-F1-model_v4.pdb... submitted as jobid=db233a618935e43. Waiting for completion......................... Done.
Submitting A

# SCanchi Notes

I used the A3D [tutorials]https://biocomp.chem.uw.edu.pl/A3D2/tutorial) on use of REST API to script the submission and result download. The original example was for one file submission and I incorporated batch logic and result parsing to ensure the submission does not overwhelm the server. I initially ran into errors parsing file for every pdb file which made me suspect structure issues with AF files. But quick inpsection of the actual pdb files clarified that the file was structured correctly and in a recognizable format. I explored the actual server side error and noticed a key error. I started with `aggrescan3Dscore` but the key is now updated to `A3Dscore`. I also updated the error log to capture the full server side error message which will always be per file basis. 

## Future improvements

The current implementation is still one file at a time submission with serial batching. For truly massive datasets, some parallel job submission balancing server loads is probably necessary. 